# View Segmentation Results

This notebook inspects outputs created by `utils/analyze_segment.py`:
- `mask/segmentation_stats.csv` (summary rows)
- saved masks & scores under `mask/masks/*`
- saved overlay PNGs under `mask/overlays/*`

Controls:
- Choose an image to inspect, then one or more features.
- By default this loads saved mask files and overlays them on the base image (fast).
- If you enable "Use SamGeo (recompute masks)", the viewer will set the image in SamGeo, optionally re-run `generate_masks()` for selected features, and display SamGeo results. Recomputing may require GPU and is slower.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import rasterio
import json
import hashlib

from pathlib import Path

def find_project_root(max_levels=4):
    p = Path.cwd()
    for _ in range(max_levels):
        if (p / "image_feature_map.json").exists() or (p / ".git").exists():
            return p
        p = p.parent
    return Path.cwd().parent  # fallback

PROJECT_ROOT = find_project_root()
CSV_PATH = PROJECT_ROOT / "mask" / "segmentation_stats.csv"

# Useful helper: deterministic color for a feature
def feature_color(feature_name: str):
    h = hashlib.sha256(feature_name.encode("utf-8")).digest()
    # Use first 3 bytes for RGB, scale to [0, 255]
    r, g, b = h[0], h[1], h[2]
    # bias to pastel-like colors
    r = int((r + 128) / 2)
    g = int((g + 128) / 2)
    b = int((b + 128) / 2)
    return (r, g, b)

def open_image_pil(path: Path):
    return Image.open(path).convert("RGBA")

def overlay_mask_on_image(img_pil: Image.Image, mask_arr: np.ndarray, color=(255, 0, 0), alpha=0.45):
    H, W = mask_arr.shape
    if img_pil.width != W or img_pil.height != H:
        img_pil = img_pil.resize((W, H), Image.LANCZOS)
    color_with_alpha = (int(color[0]), int(color[1]), int(color[2]), int(255 * alpha))
    rgba = np.zeros((H, W, 4), dtype=np.uint8)
    mask_bool = mask_arr > 0
    rgba[mask_bool, 0] = color_with_alpha[0]
    rgba[mask_bool, 1] = color_with_alpha[1]
    rgba[mask_bool, 2] = color_with_alpha[2]
    rgba[mask_bool, 3] = color_with_alpha[3]
    mask_img = Image.fromarray(rgba, mode="RGBA")
    overlay = Image.alpha_composite(img_pil, mask_img)
    return overlay

In [ ]:
if not CSV_PATH.exists():
    raise FileNotFoundError(f"CSV not found at {CSV_PATH}")

df = pd.read_csv(CSV_PATH)

# Resolve relative paths to absolute paths (PROJECT_ROOT) and try common "data/" fallback.
def resolve_path_string(ps: str):
    p = Path(ps)
    if p.is_absolute():
        return p.resolve()
    candidate = PROJECT_ROOT / ps
    if candidate.exists():
        return candidate.resolve()
    candidate2 = PROJECT_ROOT / "data" / ps
    if candidate2.exists():
        return candidate2.resolve()
    # fallback: return candidate under project, even if absent - useful for info messages
    return candidate.resolve()

def maybe_resolve(s):
    if pd.isna(s) or s == "":
        return None
    return resolve_path_string(s)

# Normalize column entries to Path objects where appropriate (resolved)
df["image_path"] = df["image"].map(lambda s: resolve_path_string(s))
df["mask_path"] = df["mask_file"].map(maybe_resolve)
df["scores_path"] = df["scores_file"].map(maybe_resolve)
df["overlay_path"] = df["overlay_file"].map(maybe_resolve)

# Build mapping: image -> list of rows (dicts) with resolved paths
grouped = {}
for _, row in df.iterrows():
    img = Path(row["image_path"])
    rec = {
        "feature": row["feature"],
        "mask_path": Path(row["mask_path"]) if row["mask_path"] is not None else None,
        "scores_path": Path(row["scores_path"]) if row["scores_path"] is not None else None,
        "overlay_path": Path(row["overlay_path"]) if row["overlay_path"] is not None else None,
        "n_objects": int(row["n_objects"]) if not pd.isna(row["n_objects"]) else None,
        "mask_pixels": int(row["mask_pixels"]) if not pd.isna(row["mask_pixels"]) else None,
        "coverage_pct": float(row["coverage_pct"]) if not pd.isna(row["coverage_pct"]) else None,
        "coverage_area_m2": float(row["coverage_area_m2"]) if not pd.isna(row["coverage_area_m2"]) else None,
        "mean_score": float(row["mean_score"]) if not pd.isna(row["mean_score"]) else None,
    }
    grouped.setdefault(img, []).append(rec)

images_list = sorted(grouped.keys())
print(f"Found {len(images_list)} images with segmentation results.")


In [ ]:
# Build the interactive widgets and callbacks (replaces the original interactive cells)
# Show readable label but keep actual dropdown values as resolved Path objects
image_dropdown = widgets.Dropdown(
    options=[((str(p.relative_to(PROJECT_ROOT)) if p.exists() else str(p)), p) for p in images_list],
    description="Image:",
    layout=widgets.Layout(width="80%")
)

def features_for_image(img_path):
    return [r["feature"] for r in grouped.get(Path(img_path), [])]

features_select = widgets.SelectMultiple(
    options=[],
    rows=10,
    description="Features:",
    layout=widgets.Layout(width="80%")
)

alpha_slider = widgets.FloatSlider(value=0.45, min=0.0, max=1.0, step=0.05, description="alpha:")
use_samgeo_checkbox = widgets.Checkbox(value=False, description="Use SamGeo (recompute masks)")
recompute_checkbox = widgets.Checkbox(value=False, description="Recompute masks (if using SamGeo)")
refresh_button = widgets.Button(description="Show", button_style="primary")

# Two separate output panes:
status_out = widgets.Output(layout={"border": "1px solid black", "max_height": "200px", "overflow": "auto"})
preview_out = widgets.Output(layout={"border": "1px solid black", "height": "512px", "overflow": "auto"})

display(image_dropdown, features_select, alpha_slider, use_samgeo_checkbox, recompute_checkbox, refresh_button, status_out, preview_out)

# sync feature options when the selected image changes
def _on_image_change(change):
    new_img = change.get("new")
    if new_img is None:
        return
    fopts = features_for_image(new_img)
    current = list(features_select.value) if features_select.value else []
    features_select.options = fopts
    restored = [f for f in current if f in fopts]
    if not restored and fopts:
        restored = [fopts[0]]
    features_select.value = tuple(restored)

# attach observer and default image
image_dropdown.observe(_on_image_change, names="value")
if images_list:
    image_dropdown.value = images_list[0]

# SamGeo lazy init with safe fallback
sam3 = None
def init_samgeo():
    global sam3
    if sam3 is not None:
        return sam3
    try:
        from samgeo import SamGeo3
    except Exception as e:
        with status_out:
            print("samgeo import failed:", e)
            print("SamGeo features will be disabled.")
        sam3 = None
        return None
    try:
        sam3 = SamGeo3(backend="transformers", device=None, checkpoint_path=None, load_from_HF=True)
    except Exception as e:
        with status_out:
            print("SamGeo initialization failed:", e)
        sam3 = None
    return sam3

def display_overlay_from_masks(base_img_path: Path, recs, alpha):
    """Load mask TIFFs and overlay them on top of the base image (fast)."""
    with preview_out:
        preview_out.clear_output(wait=True)
        if not base_img_path.exists():
            with status_out:
                print("Image file missing:", base_img_path)
                print("Make sure CSV paths are resolved correctly or that file exists.")
            return

        try:
            base_img = open_image_pil(base_img_path)
        except Exception as e:
            with status_out:
                print("Failed to open image:", base_img_path, " — ", e)
            return

        out_img = base_img
        loaded_any = False
        for rec in recs:
            mask_path = rec["mask_path"]
            if mask_path and mask_path.exists():
                try:
                    with rasterio.open(mask_path) as src:
                        arr = src.read(1)
                except Exception as e:
                    with status_out:
                        print("Failed to read mask file:", mask_path, e)
                    continue
                c = feature_color(rec["feature"])
                out_img = overlay_mask_on_image(out_img, arr, color=c, alpha=alpha)
                loaded_any = True
            else:
                if rec["overlay_path"] and rec["overlay_path"].exists():
                    try:
                        overlay_png = Image.open(rec["overlay_path"]).convert("RGBA")
                        overlay_png = overlay_png.resize(out_img.size, Image.LANCZOS)
                        out_img = Image.alpha_composite(out_img, overlay_png)
                        loaded_any = True
                    except Exception as e:
                        with status_out:
                            print("Failed to load overlay PNG:", rec["overlay_path"], e)
                        continue

        plt.figure(figsize=(10, 10))
        plt.imshow(out_img)
        plt.axis("off")
        caption = f"Overlaid features: {', '.join([r['feature'] for r in recs])}"
        plt.title(caption)
        plt.show()
        if not loaded_any:
            with status_out:
                print("No mask TIFFs or overlay PNGs were found for the selected features.")

def show_selected(_):
    with status_out:
        clear_output(wait=True)
        preview_out.clear_output(wait=True)
        img_path = Path(image_dropdown.value)
        feature_names = list(features_select.value)
        if not feature_names:
            print("No features selected — choose one or more")
            return
        if img_path not in grouped:
            print("Selected image not found in mapping:", img_path)
            return
        print(f"Image: {img_path}")
        recs = [r for r in grouped[img_path] if r["feature"] in feature_names]
        stats_df = pd.DataFrame(recs)[["feature", "n_objects", "mask_pixels", "coverage_pct", "coverage_area_m2", "mean_score"]]
        display(stats_df)

    # Show preview in the preview_out (cleared above)
    if use_samgeo_checkbox.value:
        s = init_samgeo()
        if s is None:
            with status_out:
                print("SamGeo unavailable — falling back to saved overlays.")
            display_overlay_from_masks(img_path, recs, alpha_slider.value)
            return

        try:
            s.set_image(str(img_path))
        except Exception as e:
            with status_out:
                print("SamGeo set_image failed:", e)
                print("Falling back to saved overlays.")
            display_overlay_from_masks(img_path, recs, alpha_slider.value)
            return

        if recompute_checkbox.value:
            with status_out:
                print("Recomputing masks for the selected features (this may be slow)...")
            for feature in feature_names:
                try:
                    s.generate_masks(prompt=feature)
                except Exception as e:
                    with status_out:
                        print("SamGeo generate_masks failed for", feature, ":", e)

        # show SamGeo output inside preview_out
        with preview_out:
            preview_out.clear_output(wait=True)
            try:
                s.show_masks(cmap="coolwarm")
                with status_out:
                    print("Displayed SamGeo masks.")
            except Exception as e:
                with status_out:
                    print("SamGeo show_masks failed:", e)
                    print("Falling back to saved overlays.")
                display_overlay_from_masks(img_path, recs, alpha_slider.value)
    else:
        display_overlay_from_masks(img_path, recs, alpha_slider.value)
        with status_out:
            print("Displayed overlay from saved mask TIFFs or overlay PNGs.")

# Bind the button
refresh_button.on_click(show_selected)

# Optional: update preview automatically when features change
def _on_features_change(change):
    # Only auto-show if user changes selection and a single image is selected
    if change.get("new") and image_dropdown.value:
        # don't automatically recompute SamGeo — only show saved-overlay fast preview
        display_overlay_from_masks(Path(image_dropdown.value), [r for r in grouped[Path(image_dropdown.value)] if r["feature"] in change["new"]], alpha_slider.value)

features_select.observe(_on_features_change, names="value")

In [ ]:
# Show a global stats preview
display(df.describe(include="all"))
# Provide a link to CSV file location then done.
print("CSV path:", CSV_PATH)